# Vector Prediction

Vectorized Prediction algorithms for MERCS. We need that speed.

# Imports

In [1]:
from functools import partial
import numpy as np
import numba

from mercs.utils import DESC_ENCODING, TARG_ENCODING, MISS_ENCODING, code_to_query, get_att_2d

# Functions

## Helpers

In [68]:
def criterion(
    m_matrix,
    m_filter,
    a_filter,
    aggregation=None,
):
    """
    Typical usecase 
    
    m_matrix = m_fimps
    
    m_filter = available models
    a_filter = available attributes.
    
    """
    nb_rows = m_matrix.shape[0]
    nb_cols = len(a_filter)
    
    c_matrix = np.zeros((nb_rows,nb_cols), dtype=np.int16)
    
    m_idx = a_filter + m_filter.reshape(-1,1) * m_matrix.shape[1] 
    c_matrix[m_filter, :] = m_matrix.take(m_idx.flat).reshape(nb_rows, nb_cols)
    
    #c_matrix[m_filter, :] = m_matrix[np.ix_(m_filter,a_filter)]
    #c_matrix[m_filter, :] = m_matrix[m_filter, :][:, a_filter]

    if aggregation is None: 
        return c_matrix
    else:
        return np.sum(c_matrix, axis=1).reshape(-1,1).astype(np.int16)

In [3]:
def _init_thresholds(init_threshold, stepsize):
    """Initialize thresholds array based on its two defining parameters.

    Parameters
    ----------
    init_threshold: 
    stepsize: 

    Returns
    -------

    """
    thresholds = np.round(np.arange(init_threshold, -stepsize, -stepsize), decimals=4)*10**4
    return thresholds.astype(np.int16, copy=False)

In [4]:

def pick(criteria, thresholds=None, any_target=False, stochastic=False, random_state=997):
    
    if thresholds is None:
        return np.where(criteria >= 0)[0].astype(np.int32)
    else:
        m_sel = []

        picking_function = _stochastic_pick if stochastic else _greedy_pick

        if any_target:
            criteria = np.max(criteria, axis=1).reshape(-1,1)

        for c_idx in range(criteria.shape[1]):
            m_sel.extend(picking_function(criteria[:, c_idx], thresholds=thresholds, random_state=random_state))

        return np.unique(m_sel).astype(np.int32)

def _greedy_pick(c_all, thresholds=None, **kwargs):
    for thr in thresholds:
        m_sel = np.where(c_all > thr)[0].astype(np.int32)
        if _stopping_criterion_greedy_pick(m_sel):
            break
    return m_sel

def _stochastic_pick(c_all, random_state=997, **kwargs):
    np.random.seed(random_state)
    norm = np.linalg.norm(c_all, 1)
    
    if norm > 0:
        distribution = c_all/np.sum(c_all)
    else:
        distribution = np.full(len(c_all), 1/len(c_all))
        
    draw = np.random.multinomial(1, distribution, size=1)
    return np.where(draw==1)[1].astype(np.int32)

In [5]:
def _stopping_criterion_it(q_targ, a_src):
    return np.setdiff1d(q_targ, a_src).shape[0] == 0

def _stopping_criterion_greedy_pick(m_sel):
    return len(m_sel) > 0

In [6]:
np.all(np.in1d([1], [1,2,3]))

True

## Algos

In [7]:
def mi(m_codes, m_fimps, m_score, q_code, m_avl=None, random_state=997):

    # Init
    a_src, a_tgt, _ = code_to_query(q_code)

    # Criterion
    c_all = criterion(m_score, a_filter=a_tgt, m_filter=m_avl, aggregation=None)

    # Pick
    m_sel = pick(c_all)

    return m_sel


def mrai(
    m_codes,
    m_fimps,
    m_score,
    q_code=None,
    a_src=None,
    a_tgt=None,
    m_avl=None,
    init_threshold=1.0,
    stepsize=0.1,
    any_target=False,
    stochastic=False,
    thresholds=None,
    random_state=997,
):
    # Init
    if m_avl is None:
        m_avl = np.arange(m_codes.shape[0], dtype=np.int16)
    
    if thresholds is None:
        thresholds = _init_thresholds(init_threshold, stepsize)

    if a_src is None or a_tgt is None:
        a_src, a_tgt, _ = code_to_query(q_code)

    # Criterion
    c_src = criterion(m_fimps, a_filter=a_src, m_filter=m_avl, aggregation=True)
    c_tgt = criterion(m_score, a_filter=a_tgt, m_filter=m_avl, aggregation=None)
    c_all = c_src.reshape(-1, 1) * c_tgt + c_tgt

    # Pick
    m_sel = pick(c_all, thresholds, any_target=any_target, stochastic=stochastic, random_state=random_state)

    return m_sel


def it(
    m_codes,
    m_fimps,
    m_score,
    q_code,
    m_avl=None,
    max_steps=4,
    init_threshold=1.0,
    stepsize=0.1,
    random_state=997,
):
    m_sel = []
    thresholds = _init_thresholds(init_threshold, stepsize)
    any_target = True
    stochastic = False

    q_desc, q_targ, q_miss = code_to_query(q_code)
    a_src = q_desc
    a_tgt = np.hstack([q_targ, q_miss])
    
    if m_avl is None:
        m_avl = np.arange(m_codes.shape[0], dtype=np.int16)

    for step in range(max_steps):
        
        # Check if this is our last chance
        last = step + 1 == max_steps  
        if last:
            any_target = False  # Finish the job
            a_tgt = np.setdiff1d(
                q_targ, a_src
            )  # Focus exclusively on non-predicted q_targ attributes.

        step_m_sel = mrai(
            m_codes,
            m_fimps,
            m_score,
            a_src=a_src,
            a_tgt=a_tgt,
            m_avl=m_avl,
            stochastic=stochastic,
            any_target=any_target,
            thresholds=thresholds,
            random_state=random_state,
        )

        a_prd = get_att_2d(m_codes[step_m_sel, :], kind="targ")
        
        a_src = np.union1d(a_src, a_prd)
        a_tgt = np.setdiff1d(a_tgt, a_prd)
        
        m_avl = np.setdiff1d(m_avl, step_m_sel)
        m_sel.append(step_m_sel)
        
        if _stopping_criterion_it(q_targ, a_src):
            break
            
        if len(step_m_sel) == 0:
            raise ValueError("No progress was made. This indicates an impossible query.")

    return m_sel

# Sandbox



## Setup

In [8]:
def init_m_fimps_or_m_scores(m_codes, kind='m_fimps'):
    
    if kind in {'m_fimps'}:
        value = DESC_ENCODING
    elif kind in {'m_score'}:
        value = TARG_ENCODING
    
    m_init = np.zeros(m_codes.shape)
    m_init[np.where(m_codes == value)] = np.random.rand(len(m_init[np.where(m_codes == value)]))

    normalize(m_init, norm='l1', copy=False)
    m_init = m_init*100
        
    m_init = m_init.astype(np.int8)

    return m_init

In [9]:
1.5*1.5

2.25

In [70]:
from sklearn.preprocessing import normalize

nb_attributes = 3*10**1
nb_iterations = 10**6

m_codes = np.random.randint(-1,2, size=(nb_iterations*nb_attributes, nb_attributes), dtype=np.int8)

m_fimps = init_m_fimps_or_m_scores(m_codes, kind='m_fimps')
m_score = init_m_fimps_or_m_scores(m_codes, kind='m_score')


#m_score = np.ones(m_codes.shape,  dtype=np.int8)*-1
#m_score[np.where(m_codes == 1)] = np.random.rand(len(m_score[np.where(m_codes == 1)]))
#m_score[np.where(m_codes == 1)] = 1

q_code = np.random.randint(-1,2, nb_attributes, dtype=np.int8)

#m_fimps, m_score

msg = """
m_codes:
{}
m_fimps:
{}
m_score:
{}
q_code:
{}
""".format(m_codes, m_fimps, m_score, q_code)
print(msg)


m_codes:
[[ 1  1  0 ...  0  0 -1]
 [ 1 -1 -1 ...  1  1  0]
 [ 1  1  1 ...  1  0 -1]
 ...
 [ 1  1 -1 ...  0 -1  0]
 [ 1 -1  0 ... -1  1 -1]
 [ 1  0  1 ...  0 -1  1]]
m_fimps:
[[ 0  0  4 ...  3  0  0]
 [ 0  0  0 ...  0  0 16]
 [ 0  0  0 ...  0  8  0]
 ...
 [ 0  0  0 ...  1  0 16]
 [ 0  0 21 ...  0  0  0]
 [ 0  5  0 ...  4  0  0]]
m_score:
[[ 7  4  0 ...  0  0  0]
 [11  0  0 ... 14  8  0]
 [ 9  6  4 ...  6  0  0]
 ...
 [18 14  0 ...  0  0  0]
 [ 5  0  0 ...  0 10  0]
 [ 6  0 11 ...  0  0 10]]
q_code:
[-1  0  0  0  1  1 -1  1  1  1  0  1  1  1 -1  1 -1  0  1  0 -1  1 -1  1
  1  0 -1  0 -1  0]



## Tests

In [71]:
%%timeit
it(m_codes, m_fimps, m_score, q_code, m_avl=None)

6.33 s ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%prun it(m_codes, m_fimps, m_score, q_code, m_avl=None)

         348 function calls (329 primitive calls) in 0.644 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.285    0.143    0.439    0.219 <ipython-input-52-74e74c824253>:1(criterion)
        2    0.140    0.070    0.140    0.070 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.081    0.041    0.081    0.041 {method 'take' of 'numpy.ndarray' objects}
        9    0.074    0.008    0.074    0.008 {method 'sort' of 'numpy.ndarray' objects}
        1    0.052    0.052    0.564    0.564 <ipython-input-7-1a979d9d731f>:15(mrai)
        9    0.002    0.000    0.077    0.009 arraysetops.py:297(_unique1d)
        1    0.002    0.002    0.004    0.004 <ipython-input-4-b8efb26723de>:18(_greedy_pick)
        2    0.002    0.001    0.002    0.001 {built-in method numpy.arange}
       14    0.001    0.000    0.001    0.000 {method 'astype' of 'numpy.ndarray' objects}
    40/21    0.001    0.000    0.219    0.010 {b